In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [2]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
df = pd.read_sql_query('SELECT * FROM Members', engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   memid          31 non-null     int64 
 1   surname        31 non-null     object
 2   firstname      31 non-null     object
 3   address        31 non-null     object
 4   zipcode        31 non-null     int64 
 5   telephone      31 non-null     object
 6   recommendedby  31 non-null     object
 7   joindate       31 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB


In [9]:
# Q10
q10_query = '''SELECT f.name, 
    SUM(CASE WHEN b.memid = 0 THEN slots*f.guestcost
	ELSE slots*f.membercost END) AS revenue
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
GROUP BY b.facid
HAVING revenue <1000
ORDER BY revenue'''
q10 = pd.read_sql_query(q10_query, engine)

print(q10)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [14]:
q11_query = '''SELECT main.firstname, main.surname,
	aux.firstname AS refered_by_FN, aux.surname AS refered_by_LN
FROM 
	(SELECT * FROM Members WHERE memid != 0)
        AS main
LEFT JOIN Members AS aux
ON main.recommendedby = aux.memid
ORDER BY main.surname'''
q11 = pd.read_sql_query(q11_query, engine)
print(q11)


    firstname            surname refered_by_FN refered_by_LN
0    Florence              Bader        Ponder      Stibbons
1        Anne              Baker        Ponder      Stibbons
2     Timothy              Baker        Jemima       Farrell
3         Tim             Boothe           Tim        Rownam
4      Gerald            Butters        Darren         Smith
5        Joan             Coplin       Timothy         Baker
6       Erica            Crumpet         Tracy         Smith
7       Nancy               Dare        Janice      Joplette
8      Jemima            Farrell          None          None
9       David            Farrell          None          None
10    Matthew            Genting        Gerald       Butters
11       John               Hunt     Millicent       Purview
12      David              Jones        Janice      Joplette
13    Douglas              Jones         David         Jones
14     Janice           Joplette        Darren         Smith
15       Anna          M

Q12: Find the facilities with their usage by member, but not guests

In [15]:
q12_query = '''SELECT f.name, m.firstname, m.surname, SUM(b.slots) as tot_slots
FROM Bookings AS b

LEFT JOIN Members AS m
ON m.memid = b.memid
LEFT JOIN Facilities AS f
ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY b.facid, b.memid'''
q12 = pd.read_sql_query(q12_query, engine)
print(q12)

               name  firstname            surname  tot_slots
0    Tennis Court 1      Tracy              Smith         93
1    Tennis Court 1        Tim             Rownam         18
2    Tennis Court 1     Janice           Joplette         57
3    Tennis Court 1     Gerald            Butters        171
4    Tennis Court 1     Burton              Tracy         93
..              ...        ...                ...        ...
197      Pool Table  Henrietta             Rumney          3
198      Pool Table      David            Farrell         25
199      Pool Table      Henry  Worthington-Smyth         37
200      Pool Table  Millicent            Purview          5
201      Pool Table   Hyacinth         Tupperware          9

[202 rows x 4 columns]


Q13: Find the facilities usage by month, but not guests

In [28]:
q13_query = '''SELECT f.name,
	strftime('%m',starttime) AS month,
	SUM(b.slots) as tot_slots
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY b.facid, month
ORDER BY name, month;'''
q13 = pd.read_sql_query(q13_query, engine)
print(q13)

               name month  tot_slots
0   Badminton Court    07        165
1   Badminton Court    08        414
2   Badminton Court    09        507
3    Massage Room 1    07        166
4    Massage Room 1    08        316
5    Massage Room 1    09        402
6    Massage Room 2    07          8
7    Massage Room 2    08         18
8    Massage Room 2    09         28
9        Pool Table    07        110
10       Pool Table    08        303
11       Pool Table    09        443
12    Snooker Table    07        140
13    Snooker Table    08        316
14    Snooker Table    09        404
15     Squash Court    07         50
16     Squash Court    08        184
17     Squash Court    09        184
18     Table Tennis    07         98
19     Table Tennis    08        296
20     Table Tennis    09        400
21   Tennis Court 1    07        201
22   Tennis Court 1    08        339
23   Tennis Court 1    09        417
24   Tennis Court 2    07        123
25   Tennis Court 2    08        345
2